### [Shortest Distance from all buildings](https://leetcode.com/problems/shortest-distance-from-all-buildings/)

You want to build a house on an empty land which reaches all buildings in the shortest amount of distance. You can only move up, down, left and right. You are given a 2D grid of values 0, 1 or 2, where:

Each 0 marks an empty land which you can pass by freely.
Each 1 marks a building which you cannot pass through.
Each 2 marks an obstacle which you cannot pass through.


**Example:**

```
Input: [[1,0,2,0,1],[0,0,0,0,0],[0,0,1,0,0]]

1 - 0 - 2 - 0 - 1
|   |   |   |   |
0 - 0 - 0 - 0 - 0
|   |   |   |   |
0 - 0 - 1 - 0 - 0

```
**Output:** 7 

**Explanation:** 

Given three buildings at (0,0), (0,4), (2,2), and an obstacle at (0,2), the point (1,2) is an ideal empty land to build a house, as the total travel distance of 3+3+1=7 is minimal. So return 7.

In [8]:
from collections import deque

In [9]:
class Solution(object):    
    def shortestDistance(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        
        # can view this as a tree
        # from each empty land.. we can try to reach a land that has not been previously visited
        # on the 2-D array.. if the land is empty.. we start a bfs. find the distance. update the min distance.
        
        # bfs
        #   need a tracker to keep track of visited nodes
        #   mark the node as visited
        #   can go only
        #       left
        #       right
        #       up
        #       down
        #   if lot == empty
        #       allow crossing..
        #       look for further neighbors
        #       respect grid boundaries as well
        #       add path len
        #   if lot == biulding
        #       update path len
        #       update visited building
        #       end traversal
        #   if lot == obstacle
        #       end traversal
        
        # repeat this until all nodes are visited or our node queue is empty
        #   if there are unreachable nodes, then our queue would be empty, but visited will remain < number of nodes
        # update the min distance only if all buildings are reachable.
        #   how do we if all buildings are reached? count the number of buildings in pre-processing?
        
        # bfs (grid, start, num_buildings, shortestdistance)
        #   
        
        def bfs(grid, start, num_buildings, shortest_distance):
            
            queue = deque()
            visited = set()
            
            n_rows = len(grid)
            n_cols = len(grid[0])
            
            queue.append((start, 0))
            
            building_distances = {}
            while queue:
                lot, pathlen = queue.popleft()
                
                if lot in visited:
                    continue
                
                i, j = lot    
                
                # boundary checks
                if i < 0 or i >= n_rows:
                    continue
                
                if j < 0 or j >= n_cols:
                    continue
                
                # mark this lot as visited
                visited.add(lot)
                
                if grid[i][j] == 0:
                    # empty land
                    # go visit neighbors
                    # can go left, right, up and down
                    # left = i-1, j and i > 0
                    # right = i+1, j and i < n_cols
                    # up = i, j-1 and j > 0
                    # down = i, j+1 and j < n_rows
                    left = ((i-1, j), pathlen + 1)
                    right = ((i+1, j), pathlen + 1)
                    up = ((i, j-1), pathlen + 1)
                    down = ((i, j+1), pathlen + 1)
                    
                    queue.append(left)
                    queue.append(right)
                    queue.append(up)
                    queue.append(down)
                    
                elif grid[i][j] == 1:
                    # acccumlate the distance to reach this building
                    building_distances[lot] = pathlen
                else:
                    # nothing to do.
                    continue
            
            # print("Num_Buildings: {} start: {}, distances: {}".format(num_buildings, start, building_distances))
            
            if len(building_distances) == num_buildings:
                # we visited all buildings
                shortest_distance[0] = min(shortest_distance[0], sum(building_distances.values()))
                
        
        # edge cases, later
        if not grid or not grid[0]:
            return -1
        
        shortest_distance = [float('inf')]
        
        n_rows = len(grid)
        n_cols = len(grid[0])
        
        num_buildings = 0
        for i in range(n_rows):
            for j in range(n_cols):
                if grid[i][j] == 1: # building
                    num_buildings += 1
        
        for i in range(n_rows):
            for j in range(n_cols):
                if grid[i][j] == 0: # empty land
                    bfs(grid, (i, j), num_buildings, shortest_distance)
        
        return shortest_distance[0] if shortest_distance[0] != float('inf') else -1

In [10]:
tests = {
    "test" : [
        {
            "input":{
                "grid": [[1,0,2,0,1],[0,0,0,0,0],[0,0,1,0,0]]
            },
            "output":7
        },
        {
            "input": {
                "grid": [[1,0,2,0,1]]
            },
            "output":-1
        },
        {
            "input": {
                "grid" : [[1,0,2,0,1],[0,0,0,0,1],[0,0,1,0,0],[0, 0, 0, 0, 1]]
            },
            "output":12
        },
        {
            "input": {
                "grid" : [[1,0,2,0,1],[0,0,0,0,1],[0,0,1,0,0],[1, 0, 1, 0, 1]]
            },
            "output":20
        }                
    ]
}

In [11]:
s = Solution()

for test in tests["test"]:
    inp = test["input"]
    assert(s.shortestDistance(inp["grid"]) == test["output"])

In [12]:
class Solution(object):
    def shortestDistance(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        
        # previously we started from each empty land.
        # that was rather expensive as we had to two passes and also lot BFS
        # lets reverse that logic.
        # instead of visiting all buildings from each empty land..we can visit
        # all empty lands from each building. Update the distance to reach empty
        # from every building as we do BFS. After all buildings are traversed,
        # look into our distances. if empty land was visited by all houses, then
        # update the minimum distances
        def bfs(grid, start, distances):
            
            queue = deque()
            visited = set()
            
            n_rows = len(grid)
            n_cols = len(grid[0])
            
            
            def under_limits(lot, n_rows, n_cols):
                # boundary checks
                if lot[0] < 0 or lot[0] >= n_rows:
                    return False

                if lot[1] < 0 or lot[1] >= n_cols:
                    return False

                return True
            
            
            queue.append((start, 0))
            while queue:
                lot, pathlen = queue.popleft()

                i, j = lot    
                # go to neighbor nodes
                left = (i-1, j)
                right = (i+1, j)
                up = (i, j-1)
                down = (i, j+1)
                
                for neighbor in [left, right, up, down]:
                    if under_limits(neighbor, n_rows, n_cols) and grid[neighbor[0]][neighbor[1]] == 0 and neighbor not in visited:
                        if neighbor in distances:
                            # get list of starting points
                            buildings = distances[neighbor]
                        
                            # add the distance
                            buildings[start] = pathlen + 1
                        else:
                            # visiting this node for the very first time
                            distances[neighbor] = {start : pathlen + 1}
                        
                        visited.add(neighbor)
                        
                        # we are allowed to cross only empty lands
                        queue.append((neighbor, pathlen + 1))
        
        # edge cases, later
        if not grid or not grid[0]:
            return -1
        
        n_rows = len(grid)
        n_cols = len(grid[0])
        
        num_buildings = 0
        distances = {}
        
        for i in range(n_rows):
            for j in range(n_cols):
                if grid[i][j] == 1:
                    # in a building. reach out to as many empty lot as possible.
                    # collect their distances
                    num_buildings += 1
                    bfs(grid, (i, j), distances)
    
        # once the distances are collected see if an empty land was reached out by
        # all buildings. If so update the shortest distance.
        
        shortest_distance = float('inf')
        for i in range(n_rows):
            for j in range(n_cols):
                if grid[i][j] == 0 and (i, j) in distances and len(distances[(i, j)]) == num_buildings:
                    shortest_distance = min(shortest_distance, sum(distances[(i, j)].values()))
                    
        return shortest_distance if shortest_distance != float('inf') else -1
        

In [13]:
s = Solution()

for test in tests["test"]:
    inp = test["input"]
    assert(s.shortestDistance(inp["grid"]) == test["output"])

Complexity: I think time is O(m^2 * n^2) and space is O(mn), where m - n_rows, n - n_columns